<a href="https://colab.research.google.com/github/khorzhengyu/project518/blob/main/Feature_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## One Hot Encoding ##
category_onehot = pd.get_dummies(df.category, prefix = 'category',drop_first = True)
gender_onehot = pd.get_dummies(df.gender, prefix = 'gender', drop_first = True)
day_of_week_onehot = pd.get_dummies(df.trans_day_of_week, prefix = 'day', drop_first = True)
time_of_day_onehot = pd.get_dummies(df.time_of_day, prefix = 'time_of_day', drop_first = True)
#trans_year_month_onehot = pd.get_dummies(df.trans_year_month, prefix = 'year', drop_first = True)
age_group_onehot = pd.get_dummies(df.age_group , prefix = 'age', drop_first = True)

In [ ]:
df2 = pd.concat([df, category_onehot,gender_onehot,day_of_week_onehot,time_of_day_onehot,age_group_onehot],axis = 1)

In [ ]:
df2.drop(['ccnum', 'recnum','transnum','amount_bin','merchant','street','city','state','job','category','gender','trans_day_of_week','age','lat','long','citypop','unixtime','merchlat','merchlong','trans_hour','age_group','group','time_of_day'],axis = 1, inplace = True)


In [ ]:
non_fraud_count, fraud_count = df2.isfraud.value_counts()

In [ ]:
df_num = df2.select_dtypes(include = 'number')
#df_num.drop(['index'], axis = 1, inplace = True)
df_num.columns

In [ ]:
# Data Standardiztion
from sklearn.preprocessing import RobustScaler

rob_scaler = RobustScaler()
# Scales all variables
df_scaled = rob_scaler.fit_transform(df_num)
df_num = pd.DataFrame(df_scaled, columns=df_num.columns)

In [ ]:
from sklearn.model_selection import StratifiedKFold

print('No Frauds', round(df_num['isfraud'].value_counts()[0] / len(df_num) * 100, 2), '% of the dataset')
print('Frauds', round(df_num['isfraud'].value_counts()[1] / len(df_num) * 100, 2), '% of the dataset')

X = df_num.drop('isfraud', axis=1)
y = df_num['isfraud']

sss = StratifiedKFold(n_splits=5, random_state= None , shuffle= True)

for train_index, test_index in sss.split(X, y):
    print("Train:", train_index, "Test:", test_index)
    original_Xtrain, original_Xtest = X.iloc[train_index], X.iloc[test_index]
    original_ytrain, original_ytest = y.iloc[train_index], y.iloc[test_index]


# Check the Distribution of the labels

# Turn into arrays
original_Xtrain = original_Xtrain.values
original_Xtest = original_Xtest.values
original_ytrain = original_ytrain.values
original_ytest = original_ytest.values

# See if both the train and test label distribution are similarly distributed
train_unique_label, train_counts_label = np.unique(original_ytrain, return_counts=True)
test_unique_label, test_counts_label = np.unique(original_ytest, return_counts=True)



In [ ]:
# Since our classes are highly skewed, we should make them equivalent in order to have a balanced distribution of the classes.

# Let's shuffle the data before creating the subsamples
df_num = df_num.sample(frac=1)

# Amount of fraud cases (453 rows).
fraud_df = df_num.loc[df_num['isfraud'] == 1]
non_fraud_df = df_num.loc[df_num['isfraud'] == 0][:453]  # Update the number to match your fraud cases count

balanced_df = pd.concat([fraud_df, non_fraud_df])

# Shuffle dataframe rows
new_df = balanced_df.sample(frac=1, random_state=42)

new_df.head()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

print('Distribution of the Classes in the subsample dataset')
print(new_df['isfraud'].value_counts() / len(new_df))

colors = ["#0101DF", "#DF0101"]  # Blue for non-fraud, Red for fraud

plt.figure(figsize=(7, 4))  # Adjust the figure size if needed
sns.countplot(x='isfraud', data=new_df, palette=colors)
plt.title('Equally Distributed Classes', fontsize=14)
plt.show()


In [ ]:
## Feature Selection using LASSO ##
X_cols = new_df.columns.drop('isfraud')
Y_cols = 'isfraud'

In [ ]:
# Implementing LASSO regression onto dataset
from sklearn.linear_model import LassoCV
X = pd.DataFrame(new_df[X_cols], columns=X_cols) # feature matrix
y = new_df[Y_cols] # target variable

reg = LassoCV()
reg.fit(X,y)
print("Best alpha using built_in LassoCV = %f" %reg.alpha_)
print("Best score using built-in LassoCV = %f" %reg.score(X,y))

In [ ]:
coef = pd.Series(reg.coef_, index=X.columns)
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " + str(sum(coef == 0)) + " variables")
imp_coef = coef[coef != 0].sort_values()

import matplotlib.pyplot as plt
plt.figure(figsize=(8.0, 10.0))
imp_coef.plot(kind="barh")
plt.title("Important Variables Selected by Lasso model")
plt.xlabel("Coefficient Value")
plt.ylabel("Variable")
plt.show()

picked_variables = X.columns[coef != 0]
print("Variables picked by Lasso:", picked_variables)


In [ ]:
selected_columns = ['ccnum_freq3_transdate', 'ccnum_freq30_transdate',
       'ccnum_0_dayfreq_div_14_dayfreq_velchange',
       'ccnum_0_dayfreq_div_30_dayfreq_velchange',
       'merchant_0_dayfreq_div_30_dayfreq_velchange',
       'card-merchant_daysSince', 'ccnum_totalamount1_amt2',
       'merchant_totalamount7_amt2', 'merchant_totalamount14_amt2',
       'merchant_totalamount30_amt2', 'ccnum_median0_amt2',
       'ccnum_median1_amt2', 'ccnum_median30_amt2', 'merchant_median1_amt2',
       'merchant_median7_amt2', 'merchant_median30_amt2',
       'merchant_mean0_amt2', 'merchant_mean1_amt2', 'ccnum_max1_amt2',
       'ccnum_max3_amt2', 'ccnum_max30_amt2', 'merchant_max30_amt2',
       'ccnum_median_1_actual', 'ccnum_median_3_actual',
       'ccnum_median_7_actual', 'ccnum_median_14_actual',
       'ccnum_median_30_actual', 'merchant_median_3_actual',
       'merchant_median_7_actual', 'merchant_median_14_actual',
       'merchant_median_30_actual', 'ccnum_mean_30_actual',
       'merchant_mean_3_actual', 'merchant_mean_30_actual',
       'ccnum_max_14_actual', 'merchant_max_3_actual', 'merchant_max_7_actual',
       'category_grocery_pos', 'category_shopping_net', 'day_Monday',
       'day_Wednesday', 'time_of_day_Night', 'age_61-75', 'isfraud']

df_filtered = df_num[selected_columns]